
### Atividade Prática 3 - MongoDB e BCB API

1. Extrair os dados da API do Banco Central (Atividades 3 e 4).

2. Criar um banco de dados no MongoDB Atlas e as respectivas 
coleções para armazenar os recursos retornados pela API.

3. Inserir no banco e nas coleções criadas os dados extraídos da API.

O MongoDB é um banco orientado a documentos, 
então os registros estão em formato JSON.

- Retorna os boletins diários com a Paridade de Compra e a Paridade de Venda, a Cotação de Compra e a Cotação de Venda para um período da moeda consultada. São cinco boletins para cada data, um de abertura, três intermediários e um de fechamento.

*Parâmetros*

| Nome              | Tipo     | Título        | Descrição                                                                                                                                                                |
|-------------------|----------|---------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| moeda             | texto    | Moeda         | Código texto da moeda cujas cotações e paridades serão consultadas. Formato - MMM: Três letras. Exemplo: EUR. As moedas disponíveis podem ser consultadas no recurso 'Moedas'. |
| dataInicial       | texto    | Data inicial  | Data de início do período de cotação - informar no padrão 'MM-DD-AAAA'.                                                                                                   |
| dataFinalCotacao  | texto    | Data final    | Data de fim do período de cotação - informar no padrão 'MM-DD-AAAA'.                                                                                                      |
| $format           | texto    | $format       | Tipo de conteúdo que será retornado.                                                                                                                                      |
| $select           | texto    | $select       | Propriedades que serão retornadas.                                                                                                                                        |
| $filter           | texto    | $filter       | Filtro de seleção de entidades. e.g. Nome eq 'João'. Clique aqui para as opções de operadores e funções.                                                                 |
| $orderby          | texto    | $orderby      | Propriedades para ordenação das entidades. e.g. Nome asc, Idade desc.                                                                                                     |
| $skip             | inteiro  | $skip         | Índice (maior ou igual a zero) da primeira entidade que será retornada.                                                                                                   |
| $top              | inteiro  | $top          | Número máximo (maior que zero) de entidades que serão retornadas.                                                                                                         |


*Resultado*

| Nome             | Tipo    | Título                | Descrição                                                                                                                                                        |
|------------------|---------|------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| paridadeCompra   | decimal | Paridade de compra     | Paridade de compra da moeda consultada contra o dólar: Moedas tipo A: USD/[moeda]. Moedas tipo B: [moeda]/USD.                                                   |
| paridadeVenda    | decimal | Paridade de venda      | Paridade de venda da moeda consultada contra o dólar: Moedas tipo A: USD/[moeda]. Moedas tipo B: [moeda]/USD.                                                    |
| cotacaoCompra    | decimal | Cotação de compra      | Cotação de compra da moeda consultada contra a unidade monetária corrente: unidade monetária corrente/[moeda].                                                    |
| cotacaoVenda     | decimal | Cotação de venda       | Cotação de venda da moeda consultada contra a unidade monetária corrente: unidade monetária corrente/[moeda].                                                     |
| dataHoraCotacao  | texto   | Data e hora da cotação | Data, hora e minuto das paridades e cotações.                                                                                                                      |
| tipoBoletim      | texto   | Tipo do boletim        | Tipo das paridades e cotações para aquela data e hora. Podem ser dos tipos: Abertura, Intermediário, Fechamento Interbancário ou Fechamento.                     |




In [37]:
import requests
import json
import pymongo
import pandas as pd

In [38]:
from pymongo import MongoClient

### 1. Conectando ao MongoDB Atlas
1. Entrar no MongoDB Atlas;
2. Criar um Cluster gratuito (se ainda não tiver);
3. Clicar em Connect > Connect your application.

In [ ]:
# Colar aqui a string de conexão do Atlas: NÃO COMPARTILHE SEU USUÁRIO E SENHA PELO AMOR DE ODIN
MONGO_URI = ""

# Criar conexão
client = MongoClient(MONGO_URI)

### 2. Definindo URL para requisição da API

In [40]:
url = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoMoedaPeriodo(moeda=@moeda,dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@moeda='USD'&@dataInicial='01-01-2023'&@dataFinalCotacao='09-01-2025'&$top=1000&$format=json&$select=paridadeCompra,paridadeVenda,cotacaoCompra,cotacaoVenda,dataHoraCotacao,tipoBoletim"

### 3. Criando a função para fazer a requisição e salvar o JSON na pasta Datasets

In [41]:
def requisicao_api(link):
  resposta = requests.get(link)

  if resposta.status_code == 200:      #sucesso na requisição
    dados = resposta.json()
    print('Status Code:', resposta.status_code)
    with open('datasets/usdPeriodo.json', 'w', encoding='utf-8') as arquivo:   #salvar arquivo json
      json.dump(dados, arquivo, ensure_ascii=False, indent=4)

  else:
    print('Status Code:', resposta.status_code)

### 4. Executando a função para buscar os dados

In [42]:
requisicao_api (url)

Status Code: 200


### 5. Lendo o arquivo JSON salvo e transformando em DataFrame

In [43]:
jsondata = pd.read_json('datasets/usdPeriodo.json')

dados_df = pd.json_normalize(jsondata['value'])

### 6. Visualizando as primeiras linhas do DataFrame

In [44]:
dados_df.head()

,paridadeCompra,paridadeVenda,cotacaoCompra,cotacaoVenda,dataHoraCotacao,tipoBoletim
0,1.0,1.0,5.3355,5.3361,2023-01-02 10:11:18.511,Abertura
1,1.0,1.0,5.3538,5.3544,2023-01-02 11:05:17.032,Intermediário
2,1.0,1.0,5.3466,5.3472,2023-01-02 12:10:16.594,Intermediário
3,1.0,1.0,5.3361,5.3367,2023-01-02 13:05:57.582,Intermediário
4,1.0,1.0,5.3430,5.3436,2023-01-02 13:05:57.593,Fechamento


### 7. Função para salvar o JSON numa Coleção MongoDB Atlas

In [45]:
db = client['big_data_cambio']  # Criar banco de dados
colecao = db['big_data_cambio']  # Criar a coleção

def salvar_mongodb(dados):
    registros = dados.to_dict(orient='records')  # Converter DataFrame para lista de dicionários
    colecao.insert_many(registros)  # Inserir múltiplos documentos na coleção
    print(f'{len(registros)} registros inseridos na coleção.')
salvar_mongodb(dados_df)

1000 registros inseridos na coleção.
